In [ ]:
!pip install -U  t5[gcp]\<0.8.0 tensorflow\<2.4.0 tensorflow-text\<2.4.0 numpy\<1.19.0 tfds-nightly==4.0.1.dev202011040107 tensorflow-gcs-config\<2.4.0

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import os
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = 'v2-8'
TPU_ADDRESS

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
!t5_mesh_transformer --tpu="grpc://10.100.205.10:8470" --gin_param="utils.tpu_mesh_shape.tpu_topology = 'v2-8'" \
  --model_dir="gs://supert/3B-blet" --gin_file="dataset.gin" \
  --gin_file="gs://t5-data/pretrained_models/3B/operative_config.gin" \
  --gin_param="utils.tpu_mesh_shape.model_parallelism = 8" \
  --gin_param="utils.run.train_dataset_fn = @t5.models.mesh_transformer.tsv_dataset_fn" \
  --gin_param="tsv_dataset_fn.filename = 'gs://supert/rucos-train-ru-alli.tsv'" \
  --gin_param="run.train_steps = 1005000" \
  --gin_param="run.batch_size = 128" \
  --gin_param="utils.run.save_checkpoints_steps = 500" \
  --gin_param="utils.run.learning_rate_schedule = @learning_rate_schedules.constant_learning_rate" \
  --gin_param="constant_learning_rate.learning_rate = 1e-3" \
  --gin_param="run.sequence_length = {'inputs': 768, 'targets': 64}"

In [ ]:
import t5
from google.colab import files

for step in [# 1_000_600,
             # 1_001_200,
             #1_002_400,
             # 1_001_800,,
             1_003_000,
             1_003_600,
             1_004_200,
             1_004_800
             ]:
  for model_name in ["small", "base", "large", "3B", '3B-blet', '3B'][::-1][1:2]:
    for in_file in ["val", "test", "train"][:-1]:
      model_parallelism, train_batch_size, keep_checkpoint_max = {
        "small": (1, 128, 16),
        "base": (2, 64, 8),
        "large": (8, 64, 4),
        "xl": (8, 16, 1),
        "3B": (4, 16, 1),
        "3B-alt": (8, 32, 1),
        "3B-blet": (8, 32, 1)}[model_name.replace("t5.1.1.", '')]
      model = t5.models.MtfModel(
        model_dir=f"gs://supert/{model_name}",
        tpu=TPU_ADDRESS,
        tpu_topology=TPU_TOPOLOGY,
        model_parallelism=model_parallelism,
        batch_size=train_batch_size,
        sequence_length={"inputs": 768, "targets": 64},
      )
      #model.predict(f"rucos-{in_file}.a.txt", f"zero-{model_name}.{in_file}.b.txt", temperature=0.0,
      
      #checkpoint_steps=[# 1_000_600,
      #                  # 1_001_200,
      #                  #1_001_800,
      #                  1_002_400
      #                  ])
      # !mv zero-{model_name}.{in_file}.b.txt-* zero-{model_name}.{in_file}.b.txt
      model.score(f"rucos-{in_file}.x.txt", f"rucos-{in_file}.y.txt", scores_file=f"zero-{model_name}-{step}.{in_file}",
                  checkpoint_steps=step)
      # files.download(f"{model_name}.{in_file}.b.txt")
      
      #model.score(f"rucos-{in_file}.x.txt", f"rucos-{in_file}.y.txt", scores_file=f"zero-{model_name}.{in_file}")
      #files.download(f"zero-{model_name}.{in_file}.scores")

In [ ]:
import os
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = 'v2-8'
TPU_ADDRESS

In [ ]:
!t5_mesh_transformer --tpu="grpc://10.36.72.250:8470" --gin_param="utils.tpu_mesh_shape.tpu_topology = 'v2-8'" \
  --model_dir="gs://supert/3B-alt" --gin_file="dataset.gin" \
  --gin_file="gs://t5-data/pretrained_models/3B/operative_config.gin" \
  --gin_param="utils.tpu_mesh_shape.model_parallelism = 8" \
  --gin_param="utils.run.train_dataset_fn = @t5.models.mesh_transformer.tsv_dataset_fn" \
  --gin_param="tsv_dataset_fn.filename = 'gs://supert/rucos-train-ru-zero.tsv'" \
  --gin_param="run.train_steps = 1005000" \
  --gin_param="run.batch_size = 128" \
  --gin_param="utils.run.save_checkpoints_steps = 500" \
  --gin_param="utils.run.learning_rate_schedule = @learning_rate_schedules.constant_learning_rate" \
  --gin_param="constant_learning_rate.learning_rate = 1e-3" \
  --gin_param="run.sequence_length = {'inputs': 768, 'targets': 64}"

In [ ]:
!t5_mesh_transformer --tpu="grpc://10.111.96.162:8470" --gin_param="utils.tpu_mesh_shape.tpu_topology = 'v2-8'" \
  --model_dir="gs://supert/3B" --gin_file="dataset.gin" \
  --gin_file="gs://t5-data/pretrained_models/3B/operative_config.gin" \
  --gin_param="utils.tpu_mesh_shape.model_parallelism = 8" \
  --gin_param="utils.run.train_dataset_fn = @t5.models.mesh_transformer.tsv_dataset_fn" \
  --gin_param="tsv_dataset_fn.filename = 'gs://supert/rucos-train-ru-append.tsv'" \
  --gin_param="run.train_steps = 1005000" \
  --gin_param="run.batch_size = 128" \
  --gin_param="utils.run.save_checkpoints_steps = 500" \
  --gin_param="utils.run.learning_rate_schedule = @learning_rate_schedules.constant_learning_rate" \
  --gin_param="constant_learning_rate.learning_rate = 1e-3" \
  --gin_param="run.sequence_length = {'inputs': 768, 'targets': 1}"